### This is the notebook for creating tests and performing them.

Format of the paths : p = [(x,y),...]

1st : simple, and then more complex.
Easiest : same number of points in both paths. No crossing. No going back. On segent. Same position at the beginning. Same position at the arrival.

In [7]:
# Imports

import matplotlib.pyplot as plt
import numpy as np
import unittest

import solution as sl ## the solution notebook exported as a script

SyntaxError: unexpected character after line continuation character (solution.py, line 162)

In [ ]:
# Function which displays the paths

def displ(theo,expe):
    # theo is the theoretical path
    # expe is the one really performed
    plt.clf()
    plt.figure()
    plt.plot([theo[i][0] for i in range(len(theo))],[theo[i][1] for i in range(len(theo))],c='r',label = "theoretical")
    #plt.tit("theoretical")
    plt.plot([expe[i][0] for i in range(len(expe))],[expe[i][1] for i in range(len(expe))],c='g',label = "experimental")
    #plt.label("experimental")
    plt.title("Paths")
    plt.legend()
    plt.show()

In [ ]:
# Test of the function displ.

theo = [(0,0),(1,1),(1,2)]
expe = [(0,0),(1,0),(2,0)]

displ(theo,expe)

In [ ]:
# Oracle tests
# Oracles are found on https://github.com/rouvoy/indoor-location-oracles

def oracleParser(filepath):
    """
    Text parser reading the oracle files and converting it's content to lists of tuples
    
    arg1: filepath. A string, the path to the file containing the oracle test to read.
    return: theoreticalPath. A list of tuples, the coordiantes of the theretical path.
            experimentalPath. A list of tuples, the coordiantes of the experimental path.
            expectedResult. An int or a float, the expected result computed by trajectoryError().
            error. An int or a float, the error allowed on the computation.
    """
    # define results:
    theoreticalPath = []
    experimentalPath = []
    # fetch the file
    oracle = open(file_name,mode='r')
    # extract data
    theoreticalXAxis = oracle.readline()
    theoreticalYAxis = oracle.readline()
    experimentalXAxis = oracle.readline()
    experimentalYAxis = oracle.readline()
    expectedResult = oracle.readline()
    error = oracle.readline()
    # process the strings
    theoreticalXAxis = [int(x) for x in theoreticalXAxis.split(",")]
    theoreticalYAxis = [int(x) for x in theoreticalYAxis.split(",")]
    for i in range(len(theoreticalXAxis)):
        theoreticalPath.append((theoreticalXAxis[i],theoreticalYAxis[i]))
    for i in range(len(experimentalXAxis)):
        experimentalPath.append((experimentalXAxis[i],experimentalYAxis[i]))
    expectedResult = parseNumeral(expectedResult)
    error = parseNumeral(error)
    # returns
    return theoreticalPath, experimentalPath, expectedResult, error
    
def parseNumeral(strValue):
    """
    Transforms a string value to the appropriate numeral type.
    
    arg1: strValue. A string, a numerical value.
    return: An int or a float, The value.
    """
    try:
        return int(s)
    except ValueError:
        return float(s)
    
def performOracleTest(oracleFile):
    """
    Perform oracle test
    
    arg1: oracleFile. A string, the filepath to the oracle.
    """
    
    # step 1: read the oracle and extract its data:
    theoreticalPath, experimentalPath, expectedResult, error = oracleParser(oracleFile)
    
    # step 2: compute the error on the trajectory:
    myResults = trajectoryError(theoreticalPath, experimentalPath)
    
    # step 3: compare test results:
    if myResults in range(expectedResult-error, expectedResult+error):
        ## test successfull
        print("Success")
    else:
        ## test failed
        print("Failure")
    # display
    displ(theo,expe)

__Unit tesing using the unittest.py module__

In [ ]:
class TestTrajectoryError(unittest.TestCase):
    """
    Unit testing code for the trajectoryError brick. Each test case represent a scenario with it unique theoretical and experimental trajectories.
    """

    def test_equalPaths(self): # -> Test 1
        theo = [(0,x) for x in range(5)]
        expe = [(0,x) for x in range(5)]
        #displ(theo,expe)
        
        self.assertEqual(0, sl.trajectoryError(theo, expe))# Difference expected : 0
    
    def test_equalPathsWithUTurn(self): # -> Test 2
        theo = [(0,2),(1,2),(1,0),(0,0)]
        expe = [(0,2),(1,2),(1,0),(0,0)]
        #displ(theo,expe)
        
        self.assertEqual(0, sl.trajectoryError(theo, expe))# Difference expected : 0
        
    def test_straightParallel(self): # -> Test 3
        theo = [(0,x) for x in range(5)]
        expe = [(1,x) for x in range(5)]
        #displ(theo,expe)
        
        self.assertEqual(1, sl.trajectoryError(theo, expe))# Difference expected : (area = 4 )/ (length = 4) = 1

    def test_straightCrossing(self): # -> Test 4
        theo = [(0,0),(0,1)]
        expe = [(1,0),(-1,1)]
        #displ(theo,expe)
        
        self.assertEqual(0.5, sl.trajectoryError(theo, expe))# Difference expected : 0.5
        
    def test_differentLength(self): # -> Test 5
        theo = [(0,0),(5,0)]
        expe = [(0,0)] + [(x,1) for x in range(6)] + [(5,0)]
        #displ(theo,expe)
        
        self.assertEqual(1, sl.trajectoryError(theo, expe))# Difference expected : 1

    def test_differentLengthCrossing(self): # -> Test 6
        theo = [(0,0),(0,2)]
        expe = [(0,0),(1,0),(-1,1),(1,2),(0,2)]
        #displ(theo,expe)
        
        self.assertEqual(0.75, sl.trajectoryError(theo, expe))# Difference expected : 0.75
        
    # Test 7 : when starting point and last point are the same
    
    # Test 8 : 2 crossings
    
    # Test 9 : crossing with UTurn


In [ ]:
class TestPathLength(unittest.TestCase):
    """
    Unit testing code for the path length measurement bricks. This test both the seg_length() and apth_length() functions.
    """
##    def test_typeErrorSegment(self):
##        with self.assertRaises(TypeError):
##            seg_length('a',(0,0))
##        
##    def test_typeErrorPath(self):
##        with self.assertRaises(TypeError):
##            path_length('a',(0,0))
        
    def test_simplestSegment(self):
        a = (0,0)
        b = (1,0)
        
        self.assertEqual(1, sl.seg_length(a,b))# expected 1
        
    def test_simplestSegmentInverted(self):
        a = (0,0)
        b = (1,0)
        
        self.assertEqual(1, sl.seg_length(b,a))# expected 1
        
    def test_simplestPath(self):
        path = [(0,0),(0,1),(1,1)]
        
        self.assertEqual(2, sl.path_length(path))# expected 2
        
    def test_simplestPathInverted(self):
        invPath = [(0,0),(0,-1),(-1,-1)]
        
        self.assertEqual(2, sl.path_length(invPath))# expected 2
        

In [ ]:
class TestIntersection(unittest.TestCase):
    """
    Unit testing code for the intersection locator brick.
    """

    def test_simplestIntersection(self):
        segA_1 = (0,0)
        segA_2 = (2,2)
        segB_1 = (0,2)
        segB_2 = (2,0)
        
        self.assertEqual((1,1), sl.intersection(segA_1, segA_2, segB_1, segB_2))# expected (1,1)
        
    def test_noIntersection(self):
        segA_1 = (0,0)
        segA_2 = (0,1)
        segB_1 = (1,0)
        segB_2 = (1,1)
        
        self.assertEqual(None, sl.intersection(segA_1, segA_2, segB_1, segB_2))# expected None
        
    def test_commonPoint(self):
        segA_1 = (0,0)
        segA_2 = (2,2)
        segB_1 = (0,0)
        segB_2 = (0,-1)
        
        self.assertEqual((0,0), sl.intersection(segA_1, segA_2, segB_1, segB_2))# expected (0,0)


In [ ]:
class TestOrthoProjection(unittest.TestCase):
    """
    Unit testing code for the orthogonal projection brick.
    """

    def test_simplestCase(self):
        segA_1 = (0,0)
        segA_2 = (0,2)
        y = (1,1)
        
        self.assertEqual((0,1), sl.ortogonalProjection(segA_1, segA_2, y))# expected (0,1)
        
    def test_intersectingPaths(self):
        segA_1 = (0,0)
        segA_2 = (0,2)
        y = (0,1)
        
        self.assertEqual((0,1), sl.ortogonalProjection(segA_1, segA_2, y))# expected (0,1)
        

In [ ]:
class TestComputeArea(unittest.TestCase):
    """
    Unit testing code for the area computation brick in the case of a trapezium.
    """

    def test_identicalSegments(self):
        segA_1 = (0,0)
        segA_2 = (2,2)
        segB_1 = (0,0)
        segB_2 = (2,2)
        
        self.assertEqual(0, sl.computeArea(segA_1, segA_2, segB_1, segB_2))# expected 0
        
    def test_square(self):
        segA_1 = (0,0)
        segA_2 = (0,1)
        segB_1 = (1,1)
        segB_2 = (1,0)
        
        self.assertEqual(1, sl.computeArea(segA_1, segA_2, segB_1, segB_2))# expected 1
    
    def test_triangle(self):
        segA_1 = (0,0)
        segA_2 = (0,1)
        segB_1 = (1,1)
        segB_2 = (0,0)
        
        self.assertEqual(0.5, sl.computeArea(segA_1, segA_2, segB_1, segB_2))# expected 1
    
    def test_trapezoidSimplest(self):
        segA_1 = (0,0)
        segA_2 = (0,1)
        segB_1 = (1,1)
        segB_2 = (2,0)
        
        self.assertEqual(1.5, sl.computeArea(segA_1, segA_2, segB_1, segB_2))# expected 1.5
        
    def test_trapezoidNegative(self):
        segA_1 = (0,0)
        segA_2 = (0,-1)
        segB_1 = (-1,-1)
        segB_2 = (-2,0)
        
        self.assertEqual(1.5, sl.computeArea(segA_1, segA_2, segB_1, segB_2))# expected 1.5
        
    def test_trapezoidNegativePositive(self):
        segA_1 = (-1,-1)
        segA_2 = (-1,1)
        segB_1 = (1,1)
        segB_2 = (2,-1)
        
        self.assertEqual(10.5, sl.computeArea(segA_1, segA_2, segB_1, segB_2))# expected 10.5
        

In [ ]:
class TestAreaRightTriangle(unittest.TestCase):
    """
    Unit testing for the area computation brick in the case of a right triangle
    """
    def test_flatTriangle(self):
        base = 0
        height = 3
        
        self.assertEqual(0,sl.areaRightTriangle(base,height))
    
    def test_otherFlatTriangle(self):
        base = 3
        height = 5
        
        self.assertEqual(0,sl.areaRightTriangle(base,height))
    
    def test_normalTriangle(self):
        base = 3
        height = 5
        
        self.assertEqual(15,sl.areaRightTriangle(base,height))
    
    def test_negativeDistance(self):
        base = -3
        height = 5
        
        self.assertEqual(15,sl.areaRightTriangle(base,height))
        
    def test_otherNegativeDistance(self):
        base = 3
        height = -5
        
        self.assertEqual(15,sl.areaRightTriangle(base,height))

__unittest:__ TestSuite 

In [ ]:
# initialisation
loader = unittest.TestLoader()
myTestSuite = unittest.TestSuite()
# add tests
myTestSuite.addTests(loader.loadTestsFromTestCase(TestComputeArea))
#myTestSuite.addTests(loader.loadTestFromTestCase(areaRightTriangle))
myTestSuite.addTests(loader.loadTestsFromTestCase(TestOrthoProjection))
myTestSuite.addTests(loader.loadTestsFromTestCase(TestIntersection))
myTestSuite.addTests(loader.loadTestsFromTestCase(TestPathLength))
#myTestSuite.addTests(loader.loadTestsFromTestCase(TestTrajectoryError))
# run!
runner = unittest.TextTestRunner()
runner.run(myTestSuite)